In [29]:
from urllib.request import Request
from urllib.request import urlopen
from urllib import parse
from urllib.error import HTTPError
from pprint import pprint as pp
import json
import pandas as pd
import io
import bamboolib
import time
from datetime import datetime

In [4]:
CONSUMER_KEY = "90517-93a71429963a2848be5451f9" # For Mac - https://getpocket.com/developer/apps/

ACCESS_TOKEN = "633a246a-35dd-e1e1-1fb6-c36df3"

request_url = "https://getpocket.com/v3/oauth/request"
get_url = "https://getpocket.com/v3/get"


In [5]:
def do_post_request(url, data):
    request = Request(url=url, data=data)
    try:
        http_response = urlopen(request)
    except HTTPError as e:
        print(e.headers.as_string())
    if http_response:
        text_response = http_response.read()
        return text_response

In [6]:
data = {
    'consumer_key': CONSUMER_KEY, 
    'redirect_uri': 'pocketapp1234:authorizationFinished'
}
obtain_request_token = do_post_request(request_url, parse.urlencode(data).encode())
code = obtain_request_token.decode("utf-8").split('code=')[1]
code

'02a18835-4405-b95d-8a20-6facb8'

In [40]:
data = {
    'consumer_key': CONSUMER_KEY, 
    'access_token': ACCESS_TOKEN,
    'detailType': 'complete',
    'sort': 'newest'
}
all_tags_response = do_post_request(get_url, parse.urlencode(data).encode())
all_tags = json.loads(all_tags_response.decode('utf-8'))
tags_dict = all_tags['list']

In [11]:
tags_dict

In [41]:
tags_list = []
for k,v in tags_dict.items():
    tags_list.append(v)
#tags_list

In [13]:
#tags_list[0]

In [42]:
data = pd.read_json(json.dumps(tags_list))
data

,item_id,resolved_id,given_url,given_title,favorite,status,time_added,time_updated,time_read,time_favorited,...,time_to_read,top_image_url,tags,authors,image,images,domain_metadata,listen_duration_estimate,amp_url,videos
0,5951011,5951011,https://fr.wikipedia.org/wiki/Haskell,Haskell — Wikipédia,0,0,1585837126,1585837135,0,0,...,10.0,https://upload.wikimedia.org/wikipedia/commons...,"{'informatique': {'item_id': '5951011', 'tag':...","{'52836234': {'item_id': '5951011', 'author_id...","{'item_id': '5951011', 'src': 'https://upload....","{'1': {'item_id': '5951011', 'image_id': '1', ...","{'name': 'Wikipedia', 'logo': 'https://logo.cl...",781,NaN,NaN
1,1550650088,1550650088,https://fr.wikipedia.org/wiki/Cegedim,Cegedim — Wikipédia,0,0,1585822105,1585822113,0,0,...,6.0,https://upload.wikimedia.org/wikipedia/fr/f/f8...,"{'wyde': {'item_id': '1550650088', 'tag': 'wyd...","{'52030271': {'item_id': '1550650088', 'author...","{'item_id': '1550650088', 'src': 'https://uplo...","{'1': {'item_id': '1550650088', 'image_id': '1...","{'name': 'Wikipedia', 'logo': 'https://logo.cl...",478,NaN,NaN
2,54869006,54869006,https://www.efficy.com/fr/,"Efficy, solutions CRM pour la gestion de la Re...",0,0,1585822088,1585822098,0,0,...,NaN,https://www.efficy.com/wp-content/uploads/2019...,"{'e-deal': {'item_id': '54869006', 'tag': 'e-d...","{'940964': {'item_id': '54869006', 'author_id'...",NaN,NaN,NaN,82,NaN,NaN
3,2936162599,2936162624,https://www.bdoc.com/references/partenaires/#j...,Partenaires - BDOC,0,0,1585822075,1585822082,0,0,...,NaN,https://www.bdoc.com/wp-content/uploads/2017/1...,"{'wyde': {'item_id': '2936162599', 'tag': 'wyd...",NaN,"{'item_id': '2936162599', 'src': 'https://www....","{'1': {'item_id': '2936162599', 'image_id': '1...",NaN,39,NaN,NaN
4,1145443065,1145443065,http://www.deeplearningbook.org/,Deep Learning,0,0,1585822050,1585822064,0,0,...,NaN,NaN,"{'book': {'item_id': '1145443065', 'tag': 'boo...",NaN,NaN,NaN,NaN,129,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,2064210780,2064210780,http://www.wordreference.com/enfr/go%20through,go through - English-French Dictionary WordRef...,0,0,1518086393,1518086423,0,0,...,NaN,NaN,"{'english': {'item_id': '2064210780', 'tag': '...",NaN,NaN,NaN,NaN,0,NaN,NaN
129,2064210468,2064210468,https://dictionary.cambridge.org/fr/dictionnai...,work through something définition dans le dict...,0,0,1518086368,1518086377,0,0,...,NaN,https://dictionary.cambridge.org/fr/external/i...,"{'english': {'item_id': '2064210468', 'tag': '...",NaN,NaN,NaN,NaN,8,https://dictionary.cambridge.org/fr/amp/anglai...,NaN
130,2064207864,2064207864,https://www.urbandictionary.com/define.php?ter...,Urban Dictionary: sb. ain't got shit on me,0,0,1518086173,1518783749,0,0,...,NaN,NaN,"{'expression': {'item_id': '2064207864', 'tag'...","{'83719120': {'item_id': '2064207864', 'author...",NaN,NaN,NaN,0,https://www.urbandictionary.com/define.php?ter...,NaN
131,2011690338,2011690338,http://www.marrakech-cityguide.com/site-archeo...,Site archéologique de AGHMAT à 32 km de Marrakech,0,0,1514897227,1514903673,0,0,...,NaN,http://www.marrakech-cityguide.com/wp-content/...,"{'archéologie': {'item_id': '2011690338', 'tag...","{'49416574': {'item_id': '2011690338', 'author...","{'item_id': '2011690338', 'src': 'http://www.m...","{'1': {'item_id': '2011690338', 'image_id': '1...",NaN,87,http://www.marrakech-cityguide.com/site-archeo...,NaN


In [45]:
reduced_data = pd.DataFrame(data, columns=['given_url','given_title','resolved_title'])
reduced_data = reduced_data.rename(columns={'given_url': 'lien','given_title': 'titre'})
reduced_data['date'] = data['time_added'].apply(lambda x:datetime.fromtimestamp(x).strftime('%d/%m/%y'))
reduced_data['heure'] = data['time_added'].apply(lambda x:datetime.fromtimestamp(x).strftime('%H:%M:%S'))
reduced_data['resolved_title'] = data['resolved_title']
reduced_data




,lien,titre,resolved_title,date,heure
0,https://fr.wikipedia.org/wiki/Haskell,Haskell — Wikipédia,Haskell,02/04/20,14:18:46
1,https://fr.wikipedia.org/wiki/Cegedim,Cegedim — Wikipédia,Cegedim,02/04/20,10:08:25
2,https://www.efficy.com/fr/,"Efficy, solutions CRM pour la gestion de la Re...","Efficy, Ambition 53",02/04/20,10:08:08
3,https://www.bdoc.com/references/partenaires/#j...,Partenaires - BDOC,Partenaires,02/04/20,10:07:55
4,http://www.deeplearningbook.org/,Deep Learning,Deep Learning,02/04/20,10:07:30
...,...,...,...,...,...
128,http://www.wordreference.com/enfr/go%20through,go through - English-French Dictionary WordRef...,,08/02/18,10:39:53
129,https://dictionary.cambridge.org/fr/dictionnai...,work through something définition dans le dict...,work through something définition dans le dict...,08/02/18,10:39:28
130,https://www.urbandictionary.com/define.php?ter...,Urban Dictionary: sb. ain't got shit on me,sb. ain't got shit on me,08/02/18,10:36:13
131,http://www.marrakech-cityguide.com/site-archeo...,Site archéologique de AGHMAT à 32 km de Marrakech,Site archéologique de AGHMAT,02/01/18,12:47:07
